# Simulation with Human and Cable Forces

In [3]:
import gym
from gym import spaces
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

### Pendulum Environment

This custom PendulumEnv simulates a planar pendulum influenced by both human-applied torque and external cable forces. It is built using the OpenAI Gym framework.

In [4]:
class PendulumEnv(gym.Env):
    """
    A custom pendulum environment simulating a planar pendulum with applied human_torque and an external cable force.

    State:
        [theta, theta_dot]

    Action:
        human_torque (human-applied human_torque at the pivot joint)
    """
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        super(PendulumEnv, self).__init__()
        
        # Physical and simulation parameters
        self.g = 9.81      # Gravity (m/s^2)
        self.L = 0.6       # Pendulum length (m)
        self.m = 5.0       # Pendulum mass (kg)
        self.b = 1.0       # Damping coefficient (kg*m^2/s)
        self.dt = 0.02     # Time step (s)

        # Define action and observation spaces
        high = np.array([np.pi, np.finfo(np.float32).max], dtype=np.float32)
        self.action_space = spaces.Box(low=-100.0, high=100.0, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)

        self.state = None
        self.time = 0.0
        self.cable_force = 0.0   # External force at the end-effector in the vertical direction
        self.last_torque = 0.0   # Human-applied torque in the last step

        # Viewer and plot elements
        self.viewer = None
        self.fig = None
        self.ax_pendulum = None
        self.ax_torque = None
        self.line = None
        self.force_arrow = None
        self.target_point = None
        self.theta_text = None
        self.cable_force_text = None
        self.torque_text = None

        # Data recording lists for post-run analysis
        self.time_history = []
        self.human_torque_history = []
        self.total_torque_history = []
        self.gravity_torque_history = []
        self.damping_torque_history = []
        self.cable_torque_history = []

        # Which torques to plot (user can set this)
        # Options: 'human', 'total', 'gravity', 'damping', 'cable'
        self.torque_to_plot = ['human', 'cable']  # Default plot human and total torque
    
    def set_torques_to_plot(self, torque_list):
        """
        Set which torque components to plot. 
        torque_list should be a list of strings chosen from:
        ['human', 'total', 'gravity', 'damping', 'cable'].
        Example: env.set_torques_to_plot(['human', 'gravity', 'total'])
        """
        valid_torques = {'human', 'total', 'gravity', 'damping', 'cable'}
        # Filter out invalid entries
        self.torque_to_plot = [t for t in torque_list if t in valid_torques]

    def step(self, action):
        """
        Run one timestep of the simulation.

        action: [torque]
        """
        theta, theta_dot = self.state
        human_torque = np.clip(action[0], self.action_space.low[0], self.action_space.high[0])
        Fy = self.cable_force

        # Equation of motion:
        # m*L^2 * theta_ddot = -m*g*L*sin(theta) - b*theta_dot + torque + Fy*L*sin(theta)
        # theta_ddot = (-self.g * np.sin(theta) / self.L) \
        #              - (self.b * theta_dot) / (self.m * self.L**2) \
        #              + torque / (self.m * self.L**2) \
        #              + (Fy * self.L * np.sin(theta)) / (self.m * self.L**2)
        theta_ddot = ( (-self.m*self.g/2 + Fy)*self.L*np.sin(theta) - self.b*theta_dot + human_torque ) / (self.m * self.L**2)

        # Update state using Euler integration
        theta_dot += theta_ddot * self.dt
        theta += theta_dot * self.dt

        # Normalize angle to [-pi, pi]
        theta = ((theta + np.pi) % (2 * np.pi)) - np.pi

        # Check for physical constraint (if needed)
        # For example, limit the angle to [-pi/2, pi/2]:
        max_angle = np.pi-0.05
        if theta > max_angle:
            theta = max_angle
            theta_dot = 0  # Optional: zero velocity at the limit
        elif theta < 0:
            theta = 0
            theta_dot = 0  # Optional: zero velocity at the limit

        self.state = np.array([theta, theta_dot])
        self.time += self.dt
        self.last_torque = human_torque

        # Record data for analysis
        torque_components = self.calculate_actual_torque_components()
        self.time_history.append(self.time)
        self.human_torque_history.append(torque_components['human'])
        self.cable_torque_history.append(torque_components['cable'])
        self.gravity_torque_history.append(torque_components['gravity'])
        self.damping_torque_history.append(torque_components['damping'])
        self.total_torque_history.append(torque_components['total'])
        # reward = -theta_dot**2
        done = False

        return self.state, done, {}

    def reset(self, initial_state=None):
        """
        Reset the environment to an initial state.
        If initial_state is provided, it should be [theta, theta_dot].
        Otherwise, start from zero angle and zero velocity.
        """
        if initial_state is not None:
            self.state = np.array(initial_state, dtype=np.float32)
        else:
            self.state = np.array([0.0, 0.0], dtype=np.float32)

        self.time = 0.0
        self.cable_force = 0.0
        self.last_torque = 0.0

        # Clear stored data
        self.time_history = []
        self.human_torque_history = []
        self.total_torque_history = []
        self.gravity_torque_history = []
        self.damping_torque_history = []
        self.cable_torque_history = []

        return self.state

    def render(self, mode='human'):
        """
        Render the pendulum in a 2D matplotlib plot with the current state.
        Also display chosen torque components on a subplot.
        """
        if self.viewer is None:
            plt.ion()
            # Create figure with two subplots: top for pendulum, bottom for torque
            self.fig, (self.ax_pendulum, self.ax_torque) = plt.subplots(2, 1, figsize=(5, 9))
            self.fig.subplots_adjust(hspace=0.3)
           
            # Pendulum plot setup
            self.line, = self.ax_pendulum.plot([], [], 'go-', lw=2)
            self.force_arrow = self.ax_pendulum.arrow(0, 0, 0, 0,
                                                      head_width=0.05, head_length=0.1,
                                                      fc='r', ec='r')
            self.target_point, = self.ax_pendulum.plot([], [], 'bo', markersize=5, label='MJT Target')
            self.ax_pendulum.set_xlim(-self.L - 0.5, self.L + 0.5)
            self.ax_pendulum.set_ylim(-self.L - 0.5, self.L + 0.5)
            self.ax_pendulum.set_xlabel('X (m)')
            self.ax_pendulum.set_ylabel('Y (m)')
            self.ax_pendulum.set_title('Arm Simulation')
            self.ax_pendulum.legend()

            self.viewer = True

        # Current state
        theta, theta_dot = self.state
        x = self.L * np.sin(theta)
        y = -self.L * np.cos(theta)

        # Compute target position for visualization
        target_theta = self.target_trajectory(self.time)
        target_x = [self.L * np.sin(target_theta)]
        target_y = [-self.L * np.cos(target_theta)]

        # Update pendulum line and target
        self.line.set_data([0, x], [0, y])
        self.target_point.set_data(target_x, target_y)
        if self.theta_text is None:
            self.theta_text = self.ax_pendulum.text(0.05, 0.9, '', transform=self.ax_pendulum.transAxes)
        self.theta_text.set_text('Angle = {:.2f} rad'.format(theta))

        if self.cable_force_text is None:
            self.cable_force_text = self.ax_pendulum.text(0.05, 0.85, '', transform=self.ax_pendulum.transAxes)
        self.cable_force_text.set_text('Cable Force = {:.2f} N'.format(self.cable_force))


        # Update force arrow
        self.force_arrow.remove()
        self.force_arrow = self.ax_pendulum.arrow(x, y, 0, self.cable_force / 100.0,
                                                  head_width=0.05, head_length=0.1,
                                                  fc='r', ec='r')

        # Update torque subplot
        self.ax_torque.clear()
        self.ax_torque.set_xlabel('Time (s)')
        self.ax_torque.set_ylabel('Torque (Nm)')
        self.ax_torque.set_title('Applied Torque')

        # Plot selected torques
        if len(self.time_history) > 0:
            # Create a dictionary to map torque names to data and colors
            torque_data_dict = {
                'human': (self.human_torque_history, 'g-', 'Human'),
                'total': (self.total_torque_history, 'b-', 'Total'),
                'gravity': (self.gravity_torque_history, 'b-', 'Gravity'),
                'damping': (self.damping_torque_history, 'b-', 'Damping'),
                'cable': (self.cable_torque_history, 'r-', 'Cable')
            }

            for t_name in self.torque_to_plot:
                data, style, label = torque_data_dict[t_name]
                self.ax_torque.plot(self.time_history, data, style, label=label)

            self.ax_torque.legend()

            # Update text showing current torques
            current_text = ""
            torque_components = self.calculate_actual_torque_components()
            for t_name in self.torque_to_plot:
                current_text += f"{t_name.capitalize()} Torque: {torque_components[t_name]:.2f} Nm\n"
            self.torque_text = self.ax_torque.text(0.05, 0.8, current_text.strip(), transform=self.ax_torque.transAxes)

        self.fig.canvas.draw()
        self.fig.canvas.flush_events()

    def close(self):
        if self.viewer:
            plt.ioff()
            plt.close(self.fig)
            self.viewer = None

    def target_trajectory(self, t):
        """
        Defines a minimum-jerk trajectory from 0 to 120 degrees over 5 seconds.
        """
        max_angle = 120 * (np.pi / 180)  # 120 degrees in radians
        T = 5.0  # Duration
        if t >= T:
            return max_angle
        else:
            tau = t / T
            return max_angle * (10*tau**3 - 15*tau**4 + 6*tau**5)

    def calculate_human_torque(self):
        """
        Returns the human (action) torque applied at the last step.
        """
        return self.last_torque

    def calculate_actual_torque_components(self):
        """
        Calculate and return all torque components acting on the pendulum at the current state.

        Returns a dictionary containing:
          'human':   Human-applied (action) torque
          'gravity': Torque due to gravity
          'damping': Torque due to damping
          'cable':   Torque due to cable force
          'total':   Sum of all torques
        """
        theta, theta_dot = self.state
        Fy = self.cable_force

        tau_human = self.last_torque
        tau_gravity = -self.m * self.g * self.L/2 * np.sin(theta)
        tau_damping = -self.b * theta_dot
        tau_cable = Fy * self.L * np.sin(theta)
        tau_total = tau_human + tau_gravity + tau_damping + tau_cable

        return {
            'human': tau_human,
            'gravity': tau_gravity,
            'damping': tau_damping,
            'cable': tau_cable,
            'total': tau_total
        }

    def plot_data(self):
        """
        After the simulation ends, plot the recorded time-history of chosen torques for offline analysis.
        """
        plt.figure(figsize=(6,4))
        torque_data_dict = {
            'human': (self.human_torque_history, 'g-', 'Human'),
            'total': (self.total_torque_history, 'r-', 'Total'),
            'gravity': (self.gravity_torque_history, 'b-', 'Gravity'),
            'damping': (self.damping_torque_history, 'm-', 'Damping'),
            'cable': (self.cable_torque_history, 'y-', 'Cable')
        }

        for t_name in self.torque_to_plot:
            data, style, label = torque_data_dict[t_name]
            plt.plot(self.time_history, data, style, label=label)

        plt.xlabel('Time (s)')
        plt.ylabel('Torque (Nm)')
        plt.title('Selected Torques over Time (Post-Simulation)')
        plt.legend()
        plt.grid(True)
        plt.show()

#### Example Usage

In [5]:
if __name__ == "__main__":
    env = PendulumEnv()

    # Choose which torques to visualize: e.g. human, gravity, and total
    env.set_torques_to_plot(['human', 'cable'])

    initial_state = [0.0, 0.0]  # Start at 0 deg with zero velocity
    state = env.reset(initial_state=initial_state)

    Kp = 5  # Proportional gain
    Kd = 0   # Derivative gain

    for _ in range(300):
        # Get target angle from trajectory
        target_theta = env.target_trajectory(env.time)
        current_theta, current_theta_dot = state

        # Impedance control
        error = target_theta - current_theta
        # error_dot = -current_theta_dot
        human_torque = Kp * error 
        

        # Apply human_torque as action
        action = [human_torque]

        # Set cable force if desired
        env.cable_force = 40.0

        state, done, _ = env.step(action)

        # Render environment with chosen torque lines
        env.render()
        if done:
            break

    # env.close()

    # Plot saved data after simulation
    print("Simulation ended. Backend:", matplotlib.get_backend())

Simulation ended. Backend: Qt5Agg


### Human
The `Human` class simulates human interaction with an environment using an impedance control model. The class allows for customizable parameters like proportional gain (`Kp`) and a force schedule to represent external forces applied via cables. The class also logs trajectory data during the simulation.

In [175]:
class Human:
    def __init__(self, force_schedule=None, Kp=10.0):
        self.Kp = Kp
        self.force_schedule = force_schedule
        self.trajectory_data = {
            'time': [],
            'theta': [],
            'theta_dot': [],  
            'human_torque': [],
            'cable_force': []
        }

    def impedance_torque(self, target_theta, current_theta):
        error = target_theta - current_theta
        human_torque = self.Kp * error  # Only Proportional control, Damping is handled by the environment
        return human_torque

    def simulate(self, env, cable_schedule, duration=10, render=False):
        state = env.reset(initial_state=[0.0, 0.0])

        self.trajectory_data = {
            'time': [],
            'theta': [],
            'theta_dot': [],
            'human_torque': [],
            'cable_force': []
        }   
        steps = int(duration / env.dt)
        for step in range(steps):
            env.time = step * env.dt  
            target_theta = env.target_trajectory(env.time)
            current_theta, current_theta_dot = state

            human_torque = self.impedance_torque(target_theta, current_theta)

            env.cable_force = cable_schedule.get_force(step, steps)

            # Apply torque as action
            action = [human_torque]
            state, done, _ = env.step(action)

            # Render environment if desired
            if render:
                env.render()

            # Store data for analysis
            self.trajectory_data['time'].append(env.time)
            self.trajectory_data['theta'].append(current_theta)
            self.trajectory_data['theta_dot'].append(current_theta_dot)
            self.trajectory_data['human_torque'].append(human_torque)
            self.trajectory_data['cable_force'].append(env.cable_force)

            if done:
                break

        return self.trajectory_data


### Cable Force Schedule

The CableForceSchedule class manages the application of cable forces during a simulation episode. It provides customizable force schedules, including linear, step function, and constant modes. The class calculates the cable force based on the current simulation step, the total number of steps, and the selected schedule type.

In [ ]:
class CableForceSchedule:
    """A class to manage cable force adjustments during the episode."""
    def __init__(self, initial_force=0.0, max_force=40.0, schedule_type='linear'):
        self.initial_force = initial_force
        self.max_force = max_force
        self.schedule_type = schedule_type

    def get_force(self, current_step, steps):
        """Return cable force based on current step and chosen schedule."""
        if self.schedule_type == 'constant':
            # Default to constant force
            return self.initial_force
        else:
            raise ValueError("Invalid schedule type. Choose from 'linear', 'step_function', or 'constant'.")

# Simulation and Visualization
### Human participant

In [375]:
env = PendulumEnv()
env.set_torques_to_plot(['human', 'cable'])

Superman = Human(Kp=500.0)
John = Human(Kp=50.0)
Charile = Human(Kp=5.0)

constant_0 = CableForceSchedule(initial_force=0.0, max_force=10.0, schedule_type='constant')
constant_10 = CableForceSchedule(initial_force=10.0, max_force=10.0, schedule_type='constant')
constant_40 = CableForceSchedule(initial_force=40.0, max_force=10.0, schedule_type='constant')

# Superman.simulate(env, constant_0, duration=7, render=True
# Charile.simulate(env, constant_40, duration=7, render=True)
John.simulate(env, constant_10, duration=7, render=True)

# # Now we can plot Superman's results
# data = Superman.trajectory_data
# fig, ax = plt.subplots(figsize=(10, 6))

# ax.plot(data['time'], data['theta'], label='Theta')
# ax.plot(data['time'], data['human_torque'], label='Human Torque', color='r')
# ax.plot(data['time'], data['cable_force'], label='Cable Force', color='g')

# ax.set_title("Superman's Simulation Results")
# ax.set_xlabel('Time [s]')
# ax.legend()
# plt.tight_layout()
# plt.show()


{'time': [0.02,
  0.04,
  0.06,
  0.08,
  0.1,
  0.12000000000000001,
  0.13999999999999999,
  0.16,
  0.18,
  0.19999999999999998,
  0.22,
  0.24,
  0.26,
  0.28,
  0.30000000000000004,
  0.32,
  0.34,
  0.36000000000000004,
  0.38,
  0.4,
  0.42000000000000004,
  0.44,
  0.46,
  0.48000000000000004,
  0.5,
  0.52,
  0.54,
  0.56,
  0.5800000000000001,
  0.6,
  0.62,
  0.64,
  0.66,
  0.68,
  0.7000000000000001,
  0.7200000000000001,
  0.74,
  0.76,
  0.78,
  0.8,
  0.8200000000000001,
  0.8400000000000001,
  0.86,
  0.88,
  0.9,
  0.92,
  0.9400000000000001,
  0.9600000000000001,
  0.98,
  1.0,
  1.02,
  1.04,
  1.06,
  1.08,
  1.1,
  1.12,
  1.1400000000000001,
  1.1600000000000001,
  1.18,
  1.2,
  1.22,
  1.24,
  1.26,
  1.28,
  1.3,
  1.32,
  1.34,
  1.36,
  1.3800000000000001,
  1.4000000000000001,
  1.4200000000000002,
  1.44,
  1.46,
  1.48,
  1.5,
  1.52,
  1.54,
  1.56,
  1.58,
  1.6,
  1.62,
  1.6400000000000001,
  1.6600000000000001,
  1.6800000000000002,
  1.7,
  1.72,
  

## Beysian Optimization

In [430]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# ====================================
# Your environment and human are set up
# ====================================
env = PendulumEnv()
human = Superman  # You can change this to John, Charile, or any other human object as needed

def run_simulation_and_get_error(cable_force_value, duration=7.0):
    cable_schedule = CableForceSchedule(
        initial_force=cable_force_value, 
        max_force=cable_force_value, 
        schedule_type='constant'
    )
    data = human.simulate(env, cable_schedule, duration=duration, render=False)

    thetas = np.array(data['theta'])
    times = np.array(data['time'])
    target_thetas = np.array([env.target_trajectory(t) for t in times])

    x_human = env.L * np.sin(thetas)
    y_human = -env.L * np.cos(thetas)

    x_target = env.L * np.sin(target_thetas)
    y_target = -env.L * np.cos(target_thetas)

    position_errors = (x_human - x_target)**2 + (y_human - y_target)**2
    error = np.mean(position_errors)
    return error

# Objective function now includes a penalty on the cable force effort.
def objective(**kwargs):
    cable_force = kwargs['cable_force_value']
    error = run_simulation_and_get_error(cable_force)

    # Normalize error and penalty weights
    normalized_error = error / 0.1  # Scale factor for error
    normalized_force = cable_force / 40.0  # Normalize to max force
    weight = 0.8  # Weight for penalty vs error

    cost = weight * normalized_error + (1 - weight) * normalized_force
    print("Normalized Error:", normalized_error)
    print("Normalized Force:", normalized_force)
    print("Cost:", cost)
    return cost


# ====================
# Bayesian Optimization Setup
# ====================
bounds = [(-40.0, 40.0)]
n_initial = 1
n_iterations = 7

# Initial random samples
X_init = np.random.uniform(bounds[0][0], bounds[0][1], size=(n_initial, 1))
y_init = np.array([objective(cable_force_value=x[0]) for x in X_init])

X = X_init.copy()
y = y_init.copy()

# ====================
# Define Acquisition Function (Expected Improvement)
# ====================
def expected_improvement(X_new, X, y, model, xi=0.01):
    mu, sigma = model.predict(X_new, return_std=True)
    y_min = np.min(y)

    with np.errstate(divide='warn'):
        Z = (y_min - mu - xi) / sigma
        ei = (y_min - mu - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei

def propose_next_point(acquisition, X, y, model, bounds):
    X_candidates = np.linspace(bounds[0][0], bounds[0][1], 200).reshape(-1, 1)
    ei_values = acquisition(X_candidates, X, y, model)
    best_index = np.argmax(ei_values)
    return X_candidates[best_index].reshape(-1, 1), np.max(ei_values)

# ====================
# Callback and Plotting
# ====================
fig, (ax_gp, ax_acq) = plt.subplots(2, 1, figsize=(6, 8))
plt.ion()
plt.tight_layout(pad=3.0)

def optimization_callback(iteration, X, y, model):
    print(f"Iteration {iteration}: Best so far: cable_force={X[np.argmin(y)][0]:.4f}, cost={np.min(y):.6f}")

    ax_gp.clear()
    ax_acq.clear()

    X_plot = np.linspace(bounds[0][0], bounds[0][1], 200).reshape(-1, 1)
    mu, std = model.predict(X_plot, return_std=True)

    # Plot GP mean and ±1 std
    ax_gp.plot(X_plot, mu, 'b-', label='GP Mean')
    ax_gp.fill_between(X_plot.ravel(), mu - std, mu + std, alpha=0.2, color='blue', label='GP ±1σ')

    # Plot observed data
    ax_gp.scatter(X, y, c='r', label="Observed Data")

    # Title and labels
    ax_gp.set_title("GP estimate, GP uncertainty, and Data")
    ax_gp.set_xlabel("Cable Force")
    ax_gp.set_ylabel("Cost (Normalized)")
    ax_gp.legend()

    # Plot acquisition function
    ei_values = expected_improvement(X_plot, X, y, model)
    ax_acq.plot(X_plot, ei_values, 'g-', label='Expected Improvement')
    ax_acq.set_title("Acquisition Function (EI)")
    ax_acq.set_xlabel("Cable Force")
    ax_acq.set_ylabel("Expected Improvement")
    ax_acq.legend()

    plt.pause(0.5)

# ====================
# Main Optimization Loop
# ====================
kernel = ConstantKernel(1.0) * Matern(length_scale=10.0, nu=2.5)
model = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=False, 
                                 n_restarts_optimizer=5, random_state=42)

for i in range(n_initial, n_iterations):
    model.fit(X, y)
    optimization_callback(i, X, y, model)

    # Compute next point
    X_next, ei_value = propose_next_point(expected_improvement, X, y, model, bounds)
    y_next = objective(cable_force_value=X_next[0][0])

    # Append new data
    X = np.vstack((X, X_next))
    y = np.append(y, y_next)

model.fit(X, y)
optimization_callback(n_iterations, X, y, model)

best_idx = np.argmin(y)
print("Optimization completed.")
print("Optimal cable force:", X[best_idx][0])
print("Minimum cost achieved:", y[best_idx])

# Re-run the simulation with the optimal cable force and optionally render it
optimal_cable_force = X[best_idx][0]
final_cable_schedule = CableForceSchedule(initial_force=optimal_cable_force, 
                                          max_force=optimal_cable_force, 
                                          schedule_type='constant')
final_data = human.simulate(env, final_cable_schedule, duration=7.0, render=True)
# env.close()


Normalized Error: 0.00028465065456253233
Normalized Force: 0.9086301582906969
Cost: 0.18195375218178939
Iteration 1: Best so far: cable_force=36.3452, cost=0.181954
Normalized Error: 0.01567905375777583
Normalized Force: -1.0
Cost: -0.18745675699377928
Iteration 2: Best so far: cable_force=-40.0000, cost=-0.187457


/home/awear/anaconda3/envs/gym_env/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Normalized Error: 0.015501361818025958
Normalized Force: -0.9899497487437185
Cost: -0.18558886029432292
Iteration 3: Best so far: cable_force=-40.0000, cost=-0.187457
Normalized Error: 0.01036824729699958
Normalized Force: -0.6683417085427136
Cost: -0.12537374387094302
Iteration 4: Best so far: cable_force=-40.0000, cost=-0.187457
Normalized Error: 0.01567905375777583
Normalized Force: -1.0
Cost: -0.18745675699377928
Iteration 5: Best so far: cable_force=-40.0000, cost=-0.187457
Normalized Error: 0.01567905375777583
Normalized Force: -1.0
Cost: -0.18745675699377928
Iteration 6: Best so far: cable_force=-40.0000, cost=-0.187457
Normalized Error: 0.01567905375777583
Normalized Force: -1.0
Cost: -0.18745675699377928
Iteration 7: Best so far: cable_force=-40.0000, cost=-0.187457
Optimization completed.
Optimal cable force: -40.0
Minimum cost achieved: -0.18745675699377928
